# Import Lib
Neural machine translation with a Transformer and Keras - https://www.tensorflow.org/text/tutorials/transformer#the_transformer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [15,8]
import seaborn as sns
import plotly.graph_objects as go

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Embedding, Layer, MultiHeadAttention, LayerNormalization, Conv1D
from tensorflow.keras.layers import Add, Dense, Dropout, Flatten, Concatenate, BatchNormalization
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow.math import rsqrt, minimum
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.ops import round as tf_round
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.backend import set_value
from tensorflow.signal import stft, hann_window

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error

In [ ]:
from tensorflow.keras import initializers

initializer_for_relu = initializers.HeNormal() # For layers with activation function Relu
initializer_for_sigmoid = initializers.GlorotNormal() # For layers with activation function Sigmoid

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing

## Import Data

In [ ]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/MyColabProject/Data'
data_df = pd.read_csv(path+'/raw_nsepy_inp512_differencedVal_fourierTransform.csv')
data_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,512
0,-4.45,6.45,-3.1,3.00,3.10,-2.55,-3.25,9.45,1.55,-5.2,...,-3.25,5.70,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35
1,6.45,-3.10,3.0,3.10,-2.55,-3.25,9.45,1.55,-5.20,-4.0,...,5.70,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35,2.05
2,-3.10,3.00,3.1,-2.55,-3.25,9.45,1.55,-5.20,-4.00,1.5,...,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35,2.05,1.70


In [ ]:
data_df.shape

(41412, 513)

## Train Test Split
* symbol =
           'BPCL','POWERGRID','NTPC','SUNPHARMA','TATACONSUM','ONGC','HINDALCO','ICICIBANK','SBIN','BHARTIARTL',
           'WIPRO','ITC','AXISBANK','JSWSTEEL','COALINDIA','HDFCLIFE','TATAMOTORS'
* Open, high, low, close  = 609
* Complete set = 2436

In [ ]:
data_df.iloc[:2400,256:].shape

(2400, 257)

In [ ]:
data_df.iloc[:2400,256:].head(3)

,256,257,258,259,260,261,262,263,264,265,...,503,504,505,506,507,508,509,510,511,512
0,2.75,-0.40,-1.65,-0.85,-1.0,0.30,3.90,7.85,2.75,4.30,...,-3.25,5.70,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35
1,-0.40,-1.65,-0.85,-1.00,0.3,3.90,7.85,2.75,4.30,-5.35,...,5.70,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35,2.05
2,-1.65,-0.85,-1.00,0.30,3.9,7.85,2.75,4.30,-5.35,-0.25,...,-5.45,3.20,-4.45,-2.15,-1.05,-1.10,3.05,4.35,2.05,1.70


In [ ]:
data_q1 = data_df.iloc[:2436,:129].copy()
data_q2 = data_df.iloc[:2436,128:257].copy()
data_q3 = data_df.iloc[:2436,256:385].copy()
data_q4 = data_df.iloc[:2436,384:].copy()
data_df_merged = pd.DataFrame()
data_df_merged = pd.concat([pd.DataFrame(data_q1.values),pd.DataFrame(data_q2.values),pd.DataFrame(data_q3.values),pd.DataFrame(data_q4.values)],
                           ignore_index=True)
print(data_df_merged.shape)
data_df_merged.head(3)

(9744, 129)


,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,-4.45,6.45,-3.1,3.00,3.10,-2.55,-3.25,9.45,1.55,-5.2,...,-7.65,-1.25,0.00,8.00,-6.05,1.80,-0.75,-0.9,-0.1,-2.8
1,6.45,-3.10,3.0,3.10,-2.55,-3.25,9.45,1.55,-5.20,-4.0,...,-1.25,0.00,8.00,-6.05,1.80,-0.75,-0.90,-0.1,-2.8,-1.5
2,-3.10,3.00,3.1,-2.55,-3.25,9.45,1.55,-5.20,-4.00,1.5,...,0.00,8.00,-6.05,1.80,-0.75,-0.90,-0.10,-2.8,-1.5,0.5


In [ ]:
#data_128.head(3)

In [ ]:
data_df_merged.tail(3)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
9741,7.35,1.65,0.70,0.95,-2.80,0.25,3.4,-0.50,1.30,-6.85,...,4.55,1.75,8.65,13.60,0.70,-10.25,-3.15,-0.80,-6.20,-5.75
9742,1.65,0.70,0.95,-2.80,0.25,3.40,-0.5,1.30,-6.85,-1.30,...,1.75,8.65,13.60,0.70,-10.25,-3.15,-0.80,-6.20,-5.75,-9.85
9743,0.70,0.95,-2.80,0.25,3.40,-0.50,1.3,-6.85,-1.30,-0.15,...,8.65,13.60,0.70,-10.25,-3.15,-0.80,-6.20,-5.75,-9.85,-1.00


In [ ]:
inp_len = 128 # Length of the input
out_len = 1 # Length of Output and Context

In [ ]:
prediction_percentage = 0.1 #0.025
train_df, test_df = train_test_split(data_df_merged, test_size=prediction_percentage, random_state=10, shuffle=True)

In [ ]:
print('Train shape',train_df.shape)
print('Test shape',test_df.shape)

Train shape (8769, 129)
Test shape (975, 129)


In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=10, shuffle=True)

In [ ]:
print('Train shape',train_df.shape)
print('Validation shape',val_df.shape)
print('Test shape',test_df.shape)

Train shape (6576, 129)
Validation shape (2193, 129)
Test shape (975, 129)


Keras Model.fit training expects (inputs, labels) pairs. The inputs is tokenized sequences. The labels are the same sequences shifted by 1. This shift is so that at each location input sequence, the label in the next token.

In [ ]:
X_train = train_df.drop(columns=train_df.columns[inp_len:].values).values
y_train = train_df.drop(columns=train_df.columns[:inp_len].values).values

X_val = val_df.drop(columns=val_df.columns[inp_len:].values).values
y_val = val_df.drop(columns=val_df.columns[:inp_len].values).values

X_test = test_df.drop(columns=test_df.columns[inp_len:].values).values
y_test = test_df.drop(columns=test_df.columns[:inp_len].values).values

In [ ]:
print('X Train shape',X_train.shape)
print('Y Train shape',y_train.shape)

print('X Validation shape',X_val.shape)
print('Y Validation shape',y_val.shape)

print('X Test shape',X_test.shape)
print('Y Test shape',y_test.shape)

X Train shape (6576, 128)
Y Train shape (6576, 1)
X Validation shape (2193, 128)
Y Validation shape (2193, 1)
X Test shape (975, 128)
Y Test shape (975, 1)


In [ ]:
X_train_f = X_train.astype(np.float32)
y_train_t = tf.convert_to_tensor(y_train)
X_val_f = X_val.astype(np.float32)
y_val_t = tf.convert_to_tensor(y_val)
X_test_f = X_test.astype(np.float32)
y_test_t = tf.convert_to_tensor(y_test)

In [ ]:
X_train_f[0]

array([ -6.  ,   5.75,   3.05,   2.05,  -5.7 ,  -4.  ,  10.2 ,   1.6 ,
         1.8 ,   1.75,  -3.25,   4.5 ,  -6.6 ,  -1.65, -20.95,  -1.1 ,
        -3.05,   1.85,  -1.2 ,   0.65,   8.15,   4.25,  -5.25,   3.1 ,
         2.45,   6.  ,   0.85,   0.05,  -3.5 ,   0.1 ,   3.9 ,   1.95,
        -1.  ,  -5.35,  -6.25,   2.85,   6.45,  -3.  ,   7.  ,  -1.  ,
         3.7 ,   4.85,   2.2 ,  -1.85,   3.45,   8.8 ,   6.4 ,  -5.15,
        -8.6 ,  -2.5 ,   3.25,   1.  ,   1.35,  10.15,   1.1 ,  -3.55,
         5.45,   4.25,  -4.  ,   9.2 , -11.2 ,   8.4 ,  -2.9 ,  -2.5 ,
        -2.9 ,  -5.3 ,   1.9 ,  19.9 , -43.  ,  -0.55,  18.9 ,  -1.3 ,
        14.8 ,   3.55,   1.75,   3.75,  -1.3 ,   2.65,   3.4 ,  -3.1 ,
         0.7 ,  -0.45,   3.25,  -0.85,  -0.6 ,   5.25,  -4.35,   2.8 ,
         1.05,   4.8 ,  -1.05,   4.  ,   1.25,  -2.6 ,   6.5 ,  -1.9 ,
         0.1 ,   1.3 ,  -4.55,  -5.95,  -1.5 ,  -8.4 ,   9.2 ,   0.15,
         8.05,  -0.75,   5.55,  -0.7 ,   7.55,   3.85, -12.5 ,  -2.25,
      

In [ ]:
y_train[0]

array([5.25])

In [ ]:
y_train_t[0]

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([5.25])>

## Testing tf.signal.stft

In [ ]:
spectrogram1 = tf.signal.stft(signals=X_train_f[0],
                              frame_length=512,
                              frame_step=1)

print(X_train[0].shape[0])
print(spectrogram1.shape)
print(spectrogram1)

128
(0, 257)
tf.Tensor([], shape=(0, 257), dtype=complex64)


In [ ]:
spectrogram2 = tf.signal.stft(X_train_f[0], frame_length=512, frame_step=256)

print(X_train[0].shape[0])
print(spectrogram2.shape)
print(spectrogram2)

128
(0, 257)
tf.Tensor([], shape=(0, 257), dtype=complex64)


In [ ]:
spectrogram3 = tf.signal.stft(X_train_f[0], frame_length=256, frame_step=128)

print(X_train[0].shape[0])
print(spectrogram3.shape)
print(spectrogram3)

128
(0, 129)
tf.Tensor([], shape=(0, 129), dtype=complex64)


In [ ]:
spectrogram4 = tf.signal.stft(X_train_f[0], frame_length=16, frame_step=8)

print(X_train[0].shape[0])
print(spectrogram4.shape)
print(spectrogram4)

128
(15, 9)
tf.Tensor(
[[ 8.8964295e-01 +0.j         -4.1737124e-01 -0.5630889j
  -2.2434261e+00 +1.1897726j   3.9846795e+00 -0.69102526j
  -3.5281222e+00 +0.5550227j   2.0215650e+00 +2.5805478j
   8.9342600e-01 -2.6818001j   1.1126488e-02 -2.7915158j
  -2.3333986e+00 +0.j        ]
 [-3.7691562e+00 +0.j          5.4199181e+00 +0.33517313j
  -7.4833455e+00 +2.1277974j   5.0702500e+00 -7.8546743j
  -4.0585071e-01+10.111845j   -3.7335498e+00 -6.980106j
   7.2833462e+00 +1.4282277j  -8.5566177e+00 +1.6097424j
   8.5808573e+00 +0.j        ]
 [ 2.1452644e+00 +0.j          3.1569052e+00 +1.557368j
  -2.7820301e+00 +0.43082106j -6.1971846e+00 +3.1317756j
   2.7745690e+00 -5.1250625j   1.3480473e+00 +2.5675623j
   4.0320306e+00 +0.84503496j -3.3077679e+00 +0.29315448j
  -1.9440269e-01 +0.j        ]
 [ 8.4966488e+00 +0.j         -8.3311138e+00-10.867348j
   3.1380420e+00 +7.0046477j   6.0239649e+00 +4.865336j
  -7.5904698e+00 -6.2612886j   7.8197503e-01 -2.7000442j
   1.0119579e+00 +5.0950284j  

# Model Architecture <br>
* The querys is what you're trying to find.
* The keys what sort of information the dictionary has.
* The value is that information.


<b>fft_length and frame_length</b>
* The number of frequency bins depends on the FFT size (fft_length), which is often set to the next power of 2 that is greater than or equal to frame_length
* example, if frame_lenght = 39, Next power of 2 greater than 39 is 64. Setting fft_length = 64 would allow the FFT algorithm to work more efficiently by padding the input to a length of 64.
* or choose frame_lenght any of 2, 4, 8, 16, 32, 64, ... <br>

<b>frame_step</b>
* To choose frame_step, 50% Overlap (a common choice): To achieve 50% overlap, set frame_step to half the frame_length: frame_step = frame_length/2
* example, if frame_length=8, then frame_step=8/2 = 4. This will give you overlapping frames, which improves frequency resolution.<br>

<b>Frequency_bins v/s time_bins</b>
*  have more Frequency_bins if identifying a long-term uptrend or downtrend based on weekly or monthly stock price data, or detecting cyclic behavior like seasonality.
* have more time_bins if Detecting intraday trends or anomalies (like a sudden price surge due to a news release).

In [ ]:
# Hyperparameters for Fourier Transform
sample_signal_length = X_train.shape[1] # Length of the input time series
sample_frame_length = [128, 64, 32, 16] # window size for Fourier Transform
sample_frame_size = len(sample_frame_length)
sample_frame_step = [128, 64, 32, 16] # Hop size for Fourier Transform
sample_fft_length = sample_frame_length
MAX_INPUT_SIZE = 128

# Hyperparameters for Attention Layer and DNN Layer
sample_num_layers = [1, 1, 1, 1] # number of TransformerEncoderLayer layers (Original paper = 6)
sample_num_heads = [1, 1, 1, 1] # number of self-attention heads in the MultiheadAttention layer (Original paper = 8)
sample_dropout_rate = 0.3 # Dropout rate
sample_regularizer_rate = 0.0001 # Use to regularizer the weights in attention model

# Extra Parameters
sample_index_for_testing = 1

In [ ]:
int(tf.math.floor((sample_fft_length[sample_index_for_testing]/2) +1))

33

In [ ]:
sample_fft_length[sample_index_for_testing]

64

In [ ]:
# The STFT output shape can be defined as:
# Output Shape = (batch_size, 𝐹, 𝑇)
sample_time_bins = list()
sample_frequency_bins = list()
for i in np.arange(sample_frame_size):
  print(i)
  # Number of Time Frames (T)
  sample_time_bins.append(int(((sample_signal_length - sample_fft_length[i])/sample_frame_step[i] )+1 ))
  print('Time Bins =',sample_time_bins[i])

  # Number of Frequency Bins (F)
  sample_frequency_bins.append(int(tf.math.floor((sample_fft_length[i]/2) +1)))
  print('Frequency Bins =',sample_frequency_bins[i])

  print('output_shape=[batch_size, time_bins, frequency_bins]')
  print('Output shape = (batch_size,',sample_time_bins[i],',',sample_frequency_bins[i],')\n')

0
Time Bins = 1
Frequency Bins = 65
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 1 , 65 )

1
Time Bins = 2
Frequency Bins = 33
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 2 , 33 )

2
Time Bins = 4
Frequency Bins = 17
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 4 , 17 )

3
Time Bins = 8
Frequency Bins = 9
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 8 , 9 )



In [ ]:
data_512_vals = data_df.iloc[:,:-1].values
data_512_vals

array([[ -4.45,   6.45,  -3.1 , ...,  -1.05,  -1.1 ,   3.05],
       [  6.45,  -3.1 ,   3.  , ...,  -1.1 ,   3.05,   4.35],
       [ -3.1 ,   3.  ,   3.1 , ...,   3.05,   4.35,   2.05],
       ...,
       [  4.4 ,   6.  ,  -6.1 , ..., -14.1 ,  -9.4 , -37.6 ],
       [  6.  ,  -6.1 ,  -2.55, ...,  -9.4 , -37.6 ,   4.75],
       [ -6.1 ,  -2.55, -15.25, ..., -37.6 ,   4.75,  -5.25]])

In [ ]:
data_512_vals[:,-sample_frame_length[sample_index_for_testing]:].shape

(41412, 64)

In [ ]:
data_512_vals[:,-sample_frame_length[sample_index_for_testing]:]

array([[  1.1 ,   6.8 ,   2.5 , ...,  -1.05,  -1.1 ,   3.05],
       [  6.8 ,   2.5 ,  -2.9 , ...,  -1.1 ,   3.05,   4.35],
       [  2.5 ,  -2.9 ,   4.5 , ...,   3.05,   4.35,   2.05],
       ...,
       [ -5.25,  25.25,  -9.6 , ..., -14.1 ,  -9.4 , -37.6 ],
       [ 25.25,  -9.6 ,  10.  , ...,  -9.4 , -37.6 ,   4.75],
       [ -9.6 ,  10.  ,  11.25, ..., -37.6 ,   4.75,  -5.25]])

In [ ]:
data_512_vals[:,-sample_frame_length[1]:].shape

(41412, 64)

In [ ]:
data_512_vals[:,-sample_frame_length[1]:]

array([[  1.1 ,   6.8 ,   2.5 , ...,  -1.05,  -1.1 ,   3.05],
       [  6.8 ,   2.5 ,  -2.9 , ...,  -1.1 ,   3.05,   4.35],
       [  2.5 ,  -2.9 ,   4.5 , ...,   3.05,   4.35,   2.05],
       ...,
       [ -5.25,  25.25,  -9.6 , ..., -14.1 ,  -9.4 , -37.6 ],
       [ 25.25,  -9.6 ,  10.  , ...,  -9.4 , -37.6 ,   4.75],
       [ -9.6 ,  10.  ,  11.25, ..., -37.6 ,   4.75,  -5.25]])

## The Fourier Transform layer

The output is a 2D array of shape (n_freqs, n_times), where n_freqs is the number of frequency bins (same as the length of f), and n_times is the number of time segments (same as the length of t).
* Rows of Zxx correspond to different frequencies (i.e., values in the f array).
* Columns of Zxx correspond to different time segments (i.e., values in the t array).
* The magnitude of the complex numbers in Zxx (np.abs(Zxx)) represents the strength or amplitude of each frequency component at that time segment.
* The phase of the complex numbers in Zxx (np.angle(Zxx)) represents the phase information of the signal at that frequency and time.
*  A larger window (windows_size, hops) gives better frequency resolution but poorer time resolution. You may need to experiment with different values based on your data.

* tf.py_function allows TensorFlow to execute scipy_stft_fn, passing a tensor as input, converting it to a NumPy array internally, and returning a tensor that is compatible with TensorFlow.
* In the build method of the FourierTransform class, the shape of self.kernel should be a tuple, but it is set as a single value (self.windows_size). self.windows_size should be wrapped in parentheses to specify the shape correctly, assuming it's a 1D kernel.

In [ ]:
class FourierTransform(Layer):
  def __init__(self, signal_length, frame_length, frame_step):
    super(FourierTransform, self).__init__()
    self.signal_length = signal_length
    self.frame_length = frame_length
    self.frame_step = frame_step

  def build(self, input_shape):
    # Define weights
    self.kernel = self.add_weight(
        shape=(self.signal_length,), # the use of ',' after self.signal_length is a must, read the above mentioned comments point-2
        initializer=initializer_for_relu,
        trainable=False
    )

  def call(self, x):
    #window_gen = hann_window(self.windows_size)  # symmetric Gaussian window
    # Convert the waveform to a spectrogram via a STFT.
    spectrogram = tf.signal.stft(signals=x, frame_length=self.frame_length, frame_step=self.frame_step)
    magnitude_x = tf.math.abs(spectrogram)
    angle_x = tf.math.angle(spectrogram) # Disable it if using only magnitude as output
    magnitude_x = tf_round(magnitude_x, 4)
    angle_x = tf_round(angle_x, 4) # Disable it if using only magnitude as output
    return magnitude_x, angle_x

In [ ]:
# Create an Embedding Object
sft_layer = FourierTransform(signal_length=sample_signal_length,
                             frame_length=sample_frame_length[sample_index_for_testing],
                             frame_step=sample_frame_step[sample_index_for_testing])
sft_layer.build(X_train_f[:3].shape)
# Calling the function
out_sft_mag, out_sft_ang = sft_layer(X_train_f[:3])

In [ ]:
print(out_sft_mag.shape)

(3, 2, 33)


In [ ]:
X_train_f[0,:]

array([ -6.  ,   5.75,   3.05,   2.05,  -5.7 ,  -4.  ,  10.2 ,   1.6 ,
         1.8 ,   1.75,  -3.25,   4.5 ,  -6.6 ,  -1.65, -20.95,  -1.1 ,
        -3.05,   1.85,  -1.2 ,   0.65,   8.15,   4.25,  -5.25,   3.1 ,
         2.45,   6.  ,   0.85,   0.05,  -3.5 ,   0.1 ,   3.9 ,   1.95,
        -1.  ,  -5.35,  -6.25,   2.85,   6.45,  -3.  ,   7.  ,  -1.  ,
         3.7 ,   4.85,   2.2 ,  -1.85,   3.45,   8.8 ,   6.4 ,  -5.15,
        -8.6 ,  -2.5 ,   3.25,   1.  ,   1.35,  10.15,   1.1 ,  -3.55,
         5.45,   4.25,  -4.  ,   9.2 , -11.2 ,   8.4 ,  -2.9 ,  -2.5 ,
        -2.9 ,  -5.3 ,   1.9 ,  19.9 , -43.  ,  -0.55,  18.9 ,  -1.3 ,
        14.8 ,   3.55,   1.75,   3.75,  -1.3 ,   2.65,   3.4 ,  -3.1 ,
         0.7 ,  -0.45,   3.25,  -0.85,  -0.6 ,   5.25,  -4.35,   2.8 ,
         1.05,   4.8 ,  -1.05,   4.  ,   1.25,  -2.6 ,   6.5 ,  -1.9 ,
         0.1 ,   1.3 ,  -4.55,  -5.95,  -1.5 ,  -8.4 ,   9.2 ,   0.15,
         8.05,  -0.75,   5.55,  -0.7 ,   7.55,   3.85, -12.5 ,  -2.25,
      

In [ ]:
# Inspecting the time-component when frequency-component = 3
#out_sft_mag[0,1,:]

In [ ]:
# Inspecting the time-component when frequency-component = 12
out_sft_mag[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([23.1896, 23.7   ,  1.9568, 31.4548, 36.6818,  4.0627, 15.7629,
       20.5561, 15.8429, 29.9464, 27.1647, 15.102 , 34.4287, 48.3455,
       20.5739, 20.3681, 13.0961, 22.541 , 22.0782,  7.0412,  8.8223,
       19.0087, 26.5213, 10.6311,  8.1978, 26.2701, 21.3317,  5.0147,
       19.1057,  9.6513, 23.5526, 28.0914,  9.9927], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 3
out_sft_mag[0,:,1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([23.7   , 11.5757], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 12
out_sft_mag[0,:,4]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([36.6818, 37.7549], dtype=float32)>

In [ ]:
print(out_sft_ang.shape)

(3, 2, 33)


In [ ]:
# Inspecting the time-component when frequency-component = 3
out_sft_ang[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([ 0.    ,  2.4066,  1.3112, -0.17  , -2.8917, -2.3476,  1.0521,
       -0.016 ,  2.7353, -2.1012,  0.8868, -0.9175,  2.104 , -1.2533,
        1.9649,  2.2688, -0.7524, -1.4031,  1.5528,  2.6319, -0.6408,
        2.5516, -0.8921,  2.2284, -2.0338,  0.6528, -2.0821, -3.0389,
        1.9221, -0.8739,  0.9126, -1.8802,  3.1416], dtype=float32)>

In [ ]:
# Inspecting the time-component when frequency-component = 12
#out_sft_ang[0,6,:]

In [ ]:
# Inspecting the frequency-component when time-component = 3
out_sft_ang[0,:,1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 2.4066, -2.2082], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 12
out_sft_ang[0,:,4]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.8917,  2.4298], dtype=float32)>

## Custom Attention Block

In [ ]:
class BaseAttention(Layer):
  def __init__(self, frequency_bins, time_bins, **kwargs):
    super().__init__()
    self.mha = MultiHeadAttention(key_dim = frequency_bins,
                                  kernel_initializer = initializer_for_relu,
                                  **kwargs)
    self.layernorm = LayerNormalization()
    self.add = Add()
    self.frequency_bins = frequency_bins
    self.time_bins = time_bins

  def build(self, input_shape):
    # Define weights
    self.kernel = self.add_weight(
        shape=(self.time_bins, self.frequency_bins),
        initializer=initializer_for_relu,
        trainable=True
    )

## self-attention layer - Magnitude and Angle

* tf.ensure_shape Updates the shape of a tensor and checks at runtime that the shape holds.
* When executed, this operation asserts that the input tensor x's shape is compatible with the shape argument.

In [ ]:
class SelfAttention(BaseAttention):
  def __init__(self, frequency_bins, time_bins, **kwargs):
      # Call the parent class (BaseAttention) constructor
      super().__init__(frequency_bins, time_bins, **kwargs)

  def call(self, magnitude):
    magnitude = tf.ensure_shape(magnitude, [None, self.time_bins, self.frequency_bins])
    attn_output = self.mha(
        query=magnitude,  # The querys is what you're trying to find.
        key=magnitude,  # The keys what sort of information the dictionary has.
        value=magnitude # The value is that information.
        )
    # Cache the attention scores for plotting later.
    #self.last_attn_scores = attn_scores

    x = self.add([magnitude, attn_output])
    x = self.layernorm(x)
    x = tf_round(x, 4)
    return x

In [ ]:
sample_lsa = SelfAttention(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                           time_bins=sample_time_bins[sample_index_for_testing],
                           num_heads=sample_num_heads[sample_index_for_testing],
                           dropout=sample_dropout_rate,
                           kernel_regularizer=l2(sample_regularizer_rate),
                           bias_regularizer=l2(sample_regularizer_rate),
                           activity_regularizer=l2(sample_regularizer_rate)
                                    )

In [ ]:
#sample_lsa.build(out_sft_mag.shape)
out_lsa_mag = sample_lsa(out_sft_mag)
print(out_sft_mag.shape)
print(out_lsa_mag.shape)

(3, 2, 33)
(3, 2, 33)


In [ ]:
out_lsa_mag[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([-2.1008, -0.1129, -1.1449,  1.3951,  1.1771, -0.2176,  1.1765,
        0.9819,  0.3468, -0.5939,  0.1755, -1.5823, -1.0281,  1.6769,
        0.5259,  1.8293,  0.2192,  0.6858, -1.0174,  0.3595, -1.4719,
        1.3327,  0.1678, -0.2374, -1.4512, -0.6322,  0.3977, -0.7466,
       -0.2115,  0.3103,  0.63  ,  0.3598, -1.1992], dtype=float32)>

## The global cross-attention layer

The similarity (or dot product) between the Query and each Key is computed to determine an attention score. These scores measure how relevant each item (Key) is to the current item (Query).

In [ ]:
class GlobalCrossAttention(BaseAttention):
  def __init__(self, frequency_bins, time_bins, **kwargs):
      # Call the parent class (BaseAttention) constructor
      super().__init__(frequency_bins, time_bins, **kwargs)

  def call(self, magnitude, angle):
    magnitude = tf.ensure_shape(angle, [None, self.time_bins, self.frequency_bins])
    angle = tf.ensure_shape(angle, [None, self.time_bins, self.frequency_bins])
    attn_output, attn_scores = self.mha(
        query=angle,  # The querys is what you're trying to find.
        key=magnitude,  # The keys what sort of information the dictionary has.
        value=magnitude, # The value is that information.
        return_attention_scores=True
        )

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([magnitude, attn_output])
    x = self.layernorm(x)
    x = tf_round(x, 4)
    return x

In [ ]:
sample_gca = GlobalCrossAttention(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                                  time_bins=sample_time_bins[sample_index_for_testing],
                                  num_heads=sample_num_heads[sample_index_for_testing],
                                  kernel_regularizer=l2(sample_regularizer_rate),
                                  bias_regularizer=l2(sample_regularizer_rate),
                                  activity_regularizer=l2(sample_regularizer_rate)
                                  )
#sample_gca.build(out_lsa_ang.shape)

In [ ]:
out_gca_mag = sample_gca(magnitude=out_lsa_mag, angle=out_lsa_mag)
print(out_lsa_mag.shape)
print(out_gca_mag.shape)

(3, 2, 33)
(3, 2, 33)


In [ ]:
out_gca_mag[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([-2.1837, -0.2613, -0.776 ,  0.8306,  1.5687,  1.6168, -0.8749,
        0.4147, -0.24  , -0.4846,  0.4796, -1.3936, -0.838 ,  0.4598,
       -0.3636,  2.1516,  0.7357, -0.4245, -0.8428, -0.4696, -1.3897,
        1.1402, -0.3124, -0.7748, -0.3762, -0.7597,  0.1963,  0.6126,
       -0.8861,  1.5667,  1.6164,  0.367 , -0.1053], dtype=float32)>

## Feed Forward Network Layer

In [ ]:
class FeedForward(Layer):
  # dff - dence feed forward neurons
  # sft_len - output shape Fourier Transform
  def __init__(self, frequency_bins, time_bins, regularizer_rate, dropout_rate=0.1):
    super().__init__()
    self.seq = Sequential([
        #BatchNormalization(),
        #Dense(tf.get_static_value(tf.cast(tf.math.sqrt(tf.cast(frequency_bins, dtype=tf.bfloat16))*time_bins, dtype=tf.int32)),
        #Dense(tf.get_static_value(tf.cast(frequency_bins*time_bins, dtype=tf.int32)),
        #      kernel_regularizer=l2(regularizer_rate),
        #      bias_regularizer=l2(regularizer_rate),
        #      activity_regularizer=l2(regularizer_rate),
        #      activation='relu', kernel_initializer=initializer_for_relu),
        #Dropout(dropout_rate),
        #BatchNormalization(),

        #Dense(tf.get_static_value(tf.cast(frequency_bins*time_bins, dtype=tf.int32)),
        #      kernel_regularizer=l2(regularizer_rate),
        #      bias_regularizer=l2(regularizer_rate),
        #      activity_regularizer=l2(regularizer_rate),
        #      activation='relu', kernel_initializer=initializer_for_relu),
        #Dropout(dropout_rate),
        #BatchNormalization(),

        Dense(tf.get_static_value(tf.cast(frequency_bins*time_bins, dtype=tf.int32)),
              #kernel_regularizer=l2(regularizer_rate),
              #bias_regularizer=l2(regularizer_rate),
              #activity_regularizer=l2(regularizer_rate),
              activation='relu', kernel_initializer=initializer_for_relu),
        Dropout(dropout_rate),

        #BatchNormalization(),
        Dense(frequency_bins
              #kernel_regularizer=l2(regularizer_rate),
              #bias_regularizer=l2(regularizer_rate),
              #activity_regularizer=l2(regularizer_rate)
              ) # NO Activation Function, to predict linear values as given in original paper
        #Dropout(dropout_rate)
    ])
    self.add = Add()
    self.layer_norm = LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x

In [ ]:
sample_ffn = FeedForward(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                         time_bins=sample_time_bins[sample_index_for_testing],
                         regularizer_rate=sample_regularizer_rate)
sample_ffn_out = sample_ffn(out_gca_mag)
print(out_gca_mag.shape)
print(sample_ffn_out.shape)

(3, 2, 33)
(3, 2, 33)


In [ ]:
sample_ffn_out[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([-1.0765948 ,  0.61055297,  1.2154945 ,  0.09011755,  1.3872262 ,
        1.2898489 , -1.4589895 ,  0.5189256 , -1.0202816 , -0.48315507,
        0.98051465, -1.6261293 , -2.0075252 , -0.12880434,  0.9639374 ,
        0.48832756,  0.7048087 , -1.6687655 , -0.9107037 ,  0.5091149 ,
       -1.3796477 ,  0.6919391 ,  0.38163993, -0.49688148,  0.08986323,
       -0.7006957 , -0.12271489,  0.4843266 , -1.0599104 ,  1.2954688 ,
        1.3157679 ,  1.2044623 , -0.08153862], dtype=float32)>

## Complete Encoder Layer

In [ ]:
class EncoderLayer(Layer):
  def __init__(self,*, frequency_bins, time_bins, num_heads, dropout_rate, regularizer_rate):
    super().__init__()

    self.self_attention = SelfAttention(frequency_bins=frequency_bins,
                                        time_bins=time_bins,
                                        num_heads=num_heads,
                                        dropout=dropout_rate,
                                        kernel_regularizer=l2(regularizer_rate),
                                        bias_regularizer=l2(regularizer_rate),
                                        activity_regularizer=l2(regularizer_rate)
                                             )

    self.ffn = FeedForward(frequency_bins=frequency_bins, time_bins=time_bins, regularizer_rate=regularizer_rate, dropout_rate=dropout_rate)

  def call(self, angle):
    angle = self.self_attention(angle)
    angle = self.ffn(angle)
    return angle

In [ ]:
sample_encoder_layer = EncoderLayer(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                                    time_bins=sample_time_bins[sample_index_for_testing],
                                    num_heads=sample_num_heads[sample_index_for_testing],
                                    dropout_rate=sample_dropout_rate,
                                    regularizer_rate=sample_regularizer_rate)
#sample_encoder_layer.build(out_sft_ang.shape)

In [ ]:
sample_encoder_output = sample_encoder_layer(out_sft_ang, training=False)
# Print the shape.
print(out_sft_ang.shape)
print(sample_encoder_output.shape)

(3, 2, 33)
(3, 2, 33)


In [ ]:
class Encoder(Layer):
  def __init__(self, *, frequency_bins, time_bins, num_layers, num_heads, dropout_rate, regularizer_rate):
    super().__init__()

    self.num_layers=num_layers

    self.enc_layers = [ EncoderLayer(frequency_bins=frequency_bins,
                                     time_bins=time_bins,
                                     num_heads=num_heads,
                                     dropout_rate=dropout_rate,
                                     regularizer_rate=regularizer_rate
                                     ) for _ in range(num_layers)]

  def call(self, angle):
    for i in range(self.num_layers):
      angle = self.enc_layers[i](angle)

    #self.last_attn_scores = self.enc_layers[-1].last_attn_scores
    return angle

In [ ]:
# Testing the encoder

# Instantiate the encoder.
sample_encoder = Encoder(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                         time_bins=sample_time_bins[sample_index_for_testing],
                         num_layers=sample_num_layers[sample_index_for_testing],
                         num_heads=sample_num_heads[sample_index_for_testing],
                         dropout_rate=sample_dropout_rate,
                         regularizer_rate=sample_regularizer_rate)
#sample_encoder.build(out_sft_ang.shape)

In [ ]:
sample_encoder_output = sample_encoder(out_sft_ang, training=False)
# Print the shape.
print(out_sft_ang.shape)
print(sample_encoder_output.shape)

(3, 2, 33)
(3, 2, 33)


In [ ]:
sample_encoder_output[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([ 1.4157097 , -0.10841163,  1.0294399 , -2.0224965 , -1.2204392 ,
        0.6129915 , -0.01903685, -0.4116424 ,  0.26214796,  0.71611047,
        0.6369512 ,  1.060861  , -0.81872374, -0.02388371, -0.0892847 ,
       -0.3117695 ,  0.99632823, -0.9822393 ,  0.66321915, -0.29011694,
       -0.7719937 , -0.6855173 , -1.4955653 ,  1.751625  ,  1.5461733 ,
        1.4995713 ,  0.7867392 , -0.745326  , -0.03635302, -1.6094698 ,
       -0.48383206, -1.6589582 ,  0.8071914 ], dtype=float32)>

## Complete Decoder Layer

Each DecoderLayer containing a CausalSelfAttention, a CrossAttention, and a FeedForward layer:

In [ ]:
class DecoderLayer(Layer):
  def __init__(self, *, frequency_bins, time_bins, num_heads, dropout_rate, regularizer_rate):
    super().__init__()

    self.local_self_att = SelfAttention(
        frequency_bins=frequency_bins,
        time_bins=time_bins,
        num_heads=num_heads,
        dropout=dropout_rate,
        kernel_regularizer=l2(regularizer_rate),
        bias_regularizer=l2(regularizer_rate),
        activity_regularizer=l2(regularizer_rate)
    )

    self.global_cross_att = GlobalCrossAttention(
        frequency_bins=frequency_bins,
        time_bins=time_bins,
        num_heads=num_heads,
        dropout=dropout_rate,
        kernel_regularizer=l2(regularizer_rate),
        bias_regularizer=l2(regularizer_rate),
        activity_regularizer=l2(regularizer_rate)
    )

    self.ffn = FeedForward(frequency_bins=frequency_bins, time_bins=time_bins, regularizer_rate=regularizer_rate, dropout_rate=dropout_rate)

  def call(self, magnitude, angle):
    self_att_out = self.local_self_att(magnitude)
    cross_att_out = self.global_cross_att(magnitude=self_att_out, angle=angle)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.global_cross_att.last_attn_scores

    fnn_out = self.ffn(cross_att_out)

    return fnn_out

In [ ]:
# Testing decoder layer
sample_decoder_layer = DecoderLayer(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                                    time_bins=sample_time_bins[sample_index_for_testing],
                                    num_heads=sample_num_heads[sample_index_for_testing],
                                    dropout_rate=sample_dropout_rate,
                                    regularizer_rate=sample_regularizer_rate)
#sample_decoder_layer.build(out_sft_mag.shape)
sample_decoder_layer_output = sample_decoder_layer(magnitude=out_sft_mag, angle=sample_encoder_output)

In [ ]:
print(out_sft_mag.shape)
print(sample_decoder_layer_output.shape)

(3, 2, 33)
(3, 2, 33)


In [ ]:
sample_decoder_layer_output[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([ 0.69238484,  0.5341599 ,  0.41633222, -0.2803762 , -1.2707287 ,
        0.72257984, -1.0514272 , -0.75599885, -1.3559712 , -0.56939304,
        0.24531248,  1.4739223 ,  0.65101767,  0.622545  ,  0.01024716,
        0.15799609, -0.30942178,  2.1814044 , -1.7116646 ,  0.98177767,
       -0.7497928 , -0.1872583 ,  0.38625053, -0.1802673 , -1.2679547 ,
        0.30173233, -0.9754473 , -1.3682954 , -0.09958744, -1.4097874 ,
        0.5751767 ,  1.5694804 ,  2.0210533 ], dtype=float32)>

In [ ]:
class Decoder(Layer):
  def __init__(self, *, frequency_bins, time_bins, num_heads, num_layers, dropout_rate=0.1, regularizer_rate):
    super().__init__()

    self.num_layers=num_layers

    self.decoder_layer = [ DecoderLayer(frequency_bins=frequency_bins,
                                        time_bins=time_bins,
                                        num_heads=num_heads,
                                        dropout_rate=dropout_rate,
                                        regularizer_rate=regularizer_rate
                                        ) for _ in range(num_layers)]

  def call(self, magnitude, angle):
    for i in range(self.num_layers):
      magnitude  = self.decoder_layer[i](magnitude, angle)

    self.last_attn_scores = self.decoder_layer[-1].last_attn_scores

    return magnitude

In [ ]:
# Test the decoder

# Instantiate the decoder.
sample_decoder = Decoder(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                         time_bins=sample_time_bins[sample_index_for_testing],
                         num_layers=sample_num_layers[sample_index_for_testing],
                         num_heads=sample_num_heads[sample_index_for_testing],
                         dropout_rate=sample_dropout_rate,
                         regularizer_rate=sample_regularizer_rate)
#sample_decoder.build(out_sft_mag.shape)
output = sample_decoder(magnitude=out_sft_mag, angle=sample_encoder_output)

In [ ]:
output[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([-0.33527073,  0.36999738,  0.8087568 ,  0.75132763, -0.35078624,
        0.16353545,  0.4893306 , -1.2836467 ,  2.6169016 , -0.7080039 ,
        0.30897278,  0.7826694 , -0.02168143,  0.7952994 ,  2.7032132 ,
       -0.93538165,  0.6441885 , -0.64288366, -1.1431578 ,  0.67412925,
       -0.19018793,  0.30640852, -1.2048556 ,  0.07376073,  0.29202104,
        0.81331205, -1.2234656 , -1.121707  ,  0.6440239 , -1.0101248 ,
       -1.0475066 , -1.4316257 , -0.5875633 ], dtype=float32)>

In [ ]:
print(out_sft_mag.shape)
print(sample_encoder_output.shape)
print(output.shape)

(3, 2, 33)
(3, 2, 33)
(3, 2, 33)


## Transformer

In [ ]:
class TransformerLayer(Layer):
  def __init__(self, *, signal_length, frame_length, frame_step,
               #frequency_bins, time_bins,
               num_heads, num_layers, dropout_rate=0.1, regularizer_rate):
    super().__init__()

    self.time_bins = tf.cast(((signal_length - frame_length)/frame_step)+1 , dtype=tf.int32)

    self.frequency_bins = tf.cast(tf.math.floor((frame_length/2) +1), dtype=tf.int32)

    self.filter_size = tf.get_static_value(tf.cast(tf.math.floor(signal_length/8), dtype=tf.int32))

    self.sft_layer = FourierTransform(signal_length=signal_length,
                                      frame_length=frame_length,
                                      frame_step=frame_step)

    self.encoder = Encoder(frequency_bins=tf.get_static_value(self.frequency_bins),
                           time_bins=tf.get_static_value(self.time_bins),
                           num_layers=num_layers,
                           num_heads=num_heads,
                           dropout_rate=dropout_rate,
                           regularizer_rate=regularizer_rate)

    self.decoder = Decoder(frequency_bins=tf.get_static_value(self.frequency_bins),
                           time_bins=tf.get_static_value(self.time_bins),
                           num_layers=num_layers,
                           num_heads=num_heads,
                           dropout_rate=dropout_rate,
                           regularizer_rate=regularizer_rate)

    #self.conv_layer = Conv1D(filters=self.filter_size,
    #                         kernel_size=[tf.get_static_value(self.time_bins)],
    #                         strides=[tf.get_static_value(self.time_bins)],
    #                         #activation='relu', kernel_initializer=initializer_for_relu,
    #                         input_shape = (tf.get_static_value(self.time_bins),tf.get_static_value(self.frequency_bins)))

    self.flatten_layer = Flatten()

    self.dense_layer = Dense(self.filter_size,
                              activation='relu', kernel_initializer=initializer_for_relu
                              #kernel_regularizer=l2(regularizer_rate),
                              #bias_regularizer=l2(regularizer_rate),
                              #activity_regularizer=l2(regularizer_rate)
                             )


  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the first argument.
    magnitude_x, angle_x = self.sft_layer(inputs)

    enc_out = self.encoder(angle_x)  # (batch_size, time_bins, frequency_bins)

    dec_out = self.decoder(magnitude_x, enc_out)  # (batch_size, time_bins, frequency_bins)

    # Reduce the dimentionality.
    #dec_out = self.conv_layer(dec_out) # (batch_size, 1, signal_length))
    dec_out = self.flatten_layer(dec_out)
    dec_out = self.dense_layer(dec_out)

    # Return the output and the attention weights.
    return dec_out

In [ ]:
sample_transformer_layer = TransformerLayer(signal_length=sample_signal_length,
                                            frame_length=sample_frame_length[sample_index_for_testing],
                                            frame_step=sample_frame_step[sample_index_for_testing],
                                            num_layers=sample_num_layers[sample_index_for_testing],
                                            num_heads=sample_num_heads[sample_index_for_testing],
                                            dropout_rate=sample_dropout_rate,
                                            regularizer_rate=sample_regularizer_rate)

sample_transformer_layer_output = sample_transformer_layer(X_train_f[:3])

In [ ]:
sample_transformer_layer_output.shape

TensorShape([3, 16])

## Inception

In [ ]:
class Inception(Model):
  def __init__(self, *, signal_length, frame_length, frame_step,
              #frequency_bins, time_bins,
              num_heads, num_layers, regularizer_rate, dropout_rate=0.1):
    super().__init__()
    self.iteration_len = tf.get_static_value(tf.cast(tf.size(frame_length), dtype=tf.int32))

    self.transformer = [ TransformerLayer(signal_length=signal_length, frame_length=frame_length[i], frame_step=frame_step[i],
                                        num_heads=num_heads[i],num_layers=num_layers[i], regularizer_rate=regularizer_rate,
                                        dropout_rate=dropout_rate
                                     ) for i in range(self.iteration_len)]
    self.filter_len = tf.get_static_value(tf.cast(signal_length/8, dtype=tf.int32))

    #self.conv_layer = Conv1D(filters=signal_length,
    #                         kernel_size=[self.iteration_len],
    #                         strides=[self.iteration_len],
    #                         #activation='relu', kernel_initializer=initializer_for_relu,
    #                         input_shape=(self.iteration_len, self.filter_len)
    #                         )

    #self.flatten_layer = Flatten()

    self.final_layer = Dense(1) # STUPID !!!!!! - using activation='relu' will limit the output between 0 and infinity, it won't give -ve outputs

  def call(self, inputs):
  # To use a Keras model with `.fit` you must pass all your inputs in the first argument.
    concat_out = None
    for i in range(self.iteration_len):
      transformer_out = self.transformer[i](inputs)
      if i == 0:
        concat_out = transformer_out
      else:
        concat_out = tf.concat([concat_out, transformer_out], 1)

    #concat_out = self.conv_layer(concat_out)

    #concat_out = self.flatten_layer(concat_out)

    logits = self.final_layer(concat_out)  # (batch_size, target_len)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [ ]:

sample_transformer = Inception(signal_length=sample_signal_length,
                                frame_length=sample_frame_length,
                                frame_step=sample_frame_step,
                                num_layers=sample_num_layers,
                                num_heads=sample_num_heads,
                                dropout_rate=sample_dropout_rate,
                                regularizer_rate=sample_regularizer_rate)

sample_transformer_output = sample_transformer(X_train_f[:3])
sample_transformer_output


<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-0.03325218],
       [ 0.27085674],
       [ 2.6528008 ]], dtype=float32)>

In [ ]:
#sample_transformer_output

# Model 1

## Model building

In [ ]:
#Define a model with Transformer layer
tf.keras.backend.clear_session()

In [ ]:
# Hyperparameters for Fourier Transform
signal_length = X_train.shape[1] # Length of the input time series
frame_length = [64, 32, 32, 32, 16, 16, 8, 8] # Hop size for Fourier Transform
frame_size = len(frame_length)
frame_step = [32, 32, 16, 8, 16, 8, 4, 2] # Hop size for Fourier Transform
fft_length = frame_length

# Hyperparameters for Attention Layer and DNN Layer
num_layers = [1, 1, 1, 1, 1, 1, 1, 1] # number of TransformerEncoderLayer layers (Original paper = 6)
num_heads = [1, 1, 1, 1, 1, 1, 1, 1] # number of self-attention heads in the MultiheadAttention layer (Original paper = 8)
dropout_rate = 0.3 # Dropout rate
regularizer_rate = 0.0001  # Use to regularizer the weights in attention model, smaller number for less data

In [ ]:
# The STFT output shape can be defined as:
# Output Shape = (batch_size, 𝐹, 𝑇)
time_bins = list()
frequency_bins = list()
for i in np.arange(frame_size):
  print(i)
  # Number of Time Frames (T)
  time_bins.append(int(((signal_length - fft_length[i])/frame_step[i] )+1 ))
  print('Time Bins =',time_bins[i])

  # Number of Frequency Bins (F)
  frequency_bins.append(int(tf.math.floor((fft_length[i]/2) +1)))
  print('Frequency Bins =',frequency_bins[i])

  print('output_shape=[batch_size, time_bins, frequency_bins]')
  print('Output shape = (batch_size,',time_bins[i],',',frequency_bins[i],')\n')

0
Time Bins = 4
Frequency Bins = 17
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 4 , 17 )

1
Time Bins = 7
Frequency Bins = 17
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 7 , 17 )

2
Time Bins = 13
Frequency Bins = 17
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 13 , 17 )

3
Time Bins = 8
Frequency Bins = 9
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 8 , 9 )

4
Time Bins = 15
Frequency Bins = 9
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 15 , 9 )

5
Time Bins = 31
Frequency Bins = 5
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 31 , 5 )

6
Time Bins = 61
Frequency Bins = 5
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 61 , 5 )



In [ ]:
transformer = Inception(signal_length=signal_length,
                        frame_length=frame_length,
                        frame_step=frame_step,
                        num_layers=num_layers,
                        num_heads=num_heads,
                        dropout_rate=dropout_rate,
                        regularizer_rate=regularizer_rate)

In [ ]:
#transformer.build(X_train_f.shape)

In [ ]:
transformer_output = transformer(X_train_f[:3])

print(X_train_f[:3].shape)
print(transformer_output.shape)

(3, 128)
(3, 1)


In [ ]:
transformer.summary()

Model: "inception"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ transformer_layer (TransformerLayer) │ ?                           │          10,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_1                  │ ?                           │          14,611 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_2                  │ ?                           │          23,689 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_3                  │ ?                           │           6,489 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_4                  │ ?                           │          10,080 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_5                  │ ?                           │           7,264 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_6                  │ ?                           │          13,414 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (3, 1)                      │             113 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 85,732 (334.89 KB)

 Trainable params: 84,836 (331.39 KB)

 Non-trainable params: 896 (3.50 KB)

In [ ]:
Model: "inception"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ transformer_layer (TransformerLayer) │ ?                           │          70,669 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_1                  │ ?                           │          24,102 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_2                  │ ?                           │          29,151 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_3                  │ ?                           │          10,072 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_4                  │ ?                           │           5,436 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (3, 1)                      │              81 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
 Total params: 139,511 (544.96 KB)
 Trainable params: 138,871 (542.46 KB)
 Non-trainable params: 640 (2.50 KB)

## Training

In [ ]:
# Define the learning rate variable
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-8, clipnorm=1.0)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=2)

In [ ]:
transformer.compile(
    loss='mean_squared_error',#'mean_absolute_error',
    optimizer= optimizer,
    metrics=['R2Score'])# 'mean_squared_error'

In [ ]:
path_model = '/content/drive/MyDrive/MyColabProject/Models'
checkpoint_filepath = (path_model+'/Idea1_1SF_nsepy_Model_inp128_out1_fourierTransform_withInception.weights.h5')
print(checkpoint_filepath)

/content/drive/MyDrive/MyColabProject/Models/Idea1_1SF_nsepy_Model_inp128_out1_fourierTransform_withInception.weights.h5


In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

early_stopping = EarlyStopping(monitor="val_loss",
                               patience=10,
                               mode="min",
                               restore_best_weights=True)

In [ ]:
history_1 = transformer.fit(X_train_f,y_train,
                            epochs=50, batch_size=32,
                            shuffle = True,
                            validation_data=[X_val_f,y_val],
                            callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

Epoch 1/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 226s 522ms/step - R2Score: -0.0099 - loss: 2101.6304 - val_R2Score: 0.0184 - val_loss: 370.7587 - learning_rate: 0.0010
Epoch 2/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 47s 36ms/step - R2Score: 0.0194 - loss: 271.0278 - val_R2Score: 0.0215 - val_loss: 99.4494 - learning_rate: 0.0010
Epoch 3/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - R2Score: 0.0609 - loss: 75.4743 - val_R2Score: 0.0627 - val_loss: 33.2771 - learning_rate: 0.0010
Epoch 4/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - R2Score: 0.0927 - loss: 29.4299 - val_R2Score: 0.0826 - val_loss: 18.3281 - learning_rate: 0.0010
Epoch 5/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - R2Score: 0.1307 - loss: 18.1999 - val_R2Score: 0.0889 - val_loss: 16.3248 - learning_rate: 0.0010
Epoch 6/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - R2Score: 0.1910 - loss: 15.0756 - val_R2Score: 0.1183 - val_loss: 15.6434 - learning_rate: 0.0010
Epoch 7/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - R2Score: 0.2134

In [ ]:
history_2 = transformer.fit(X_train_f,y_train,
                            initial_epoch=50, epochs=100,
                            batch_size=32, shuffle = True,
                            validation_data=[X_val_f,y_val],
                            callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

In [ ]:
history_3 = transformer.fit(X_train_f,y_train,
                            initial_epoch=100, epochs=150,
                            batch_size=32, shuffle = True,
                            validation_data=[X_val_f,y_val],
                            callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

In [ ]:
# Train Loss
train_loss = list(history_1.history['loss'])
#train_loss.extend(history_2.history['loss'])
#train_loss.extend(history_3.history['loss'])
#train_loss.extend(history_4.history['loss'])

# Validation Loss
val_loss = list(history_1.history['val_loss'])
#val_loss.extend(history_2.history['val_loss'])
#val_loss.extend(history_3.history['val_loss'])
#val_loss.extend(history_4.history['val_loss'])

# plot history
plt.plot(np.array(train_loss[5:]), label='train loss')
plt.plot(np.array(val_loss[5:]), label='validation loss')
plt.xlabel("Number of Epochs ")
plt.ylabel("Loss")
plt.grid()
plt.legend()
plt.show()

## Prediction - from Transformer

In [ ]:
predictions = transformer((X_test_f), training=False)
#predictions = transformer((X_train_f), training=False)

In [ ]:
predictions.shape

TensorShape([975, 1])

In [ ]:
predictions[0]

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.2819138], dtype=float32)>

In [ ]:
i = 12
print('Predictions = \n',predictions[i])
print('Actuals = \n',y_test[i])

Predictions = 
 tf.Tensor([0.37353134], shape=(1,), dtype=float32)
Actuals = 
 -2.4499999999999886


### Validating the results

In [ ]:
y_pred = predictions.numpy().reshape(-1)
y_test = y_test.reshape(-1)
#y_train = y_train.reshape(-1)
print(y_pred.shape)
print(y_test.shape)

(975,)
(975,)


In [ ]:
score_mae = mean_absolute_error(y_test, y_pred) # y_test
print("The Mean Absolute Error of our Model is {}".format(round(score_mae, 2)))
score_rmse = mean_squared_error(y_test, y_pred)
print("The Root Mean Squared Error of our Model is {}".format(round(score_rmse, 2)))
score_r2 = r2_score(y_test, y_pred)
print("The accuracy of our model is {}%".format(round(score_r2, 2) *100))

The Mean Absolute Error of our Model is 1.01
The Root Mean Squared Error of our Model is 5.27
The accuracy of our model is 59.0%


In [ ]:
fig = go.Figure(data=[go.Ohlc(x=np.arange(0,10000,1),
                    open=y_pred,
                    high=y_pred,
                    low=y_test,
                    close=y_test
                              )])
fig.show()

In [ ]:
'''
fig = go.Figure(data=[go.Ohlc(x=np.arange(0,1000,1),
                    open=y_pred,
                    high=y_pred,
                    low=y_test,
                    close=y_test
                              )])
fig.show()
'''

'\nfig = go.Figure(data=[go.Ohlc(x=np.arange(0,1000,1),\n                    open=y_pred,\n                    high=y_pred,\n                    low=y_test,\n                    close=y_test\n                              )])\nfig.show()\n'

 ## Prediction - from Saved model

In [ ]:
# Save the model - not required - using different save method

#model_path = '/content/drive/MyDrive/hobby_project/Models/forecaster_future1_pred_finetune/'
#tf.saved_model.save(osf_tf_function, export_dir=model_path)

In [ ]:
# Load the model
model_path = '/content/drive/MyDrive/MyColabProject/Models/Idea1_1SF_nsepy_Model_inp128_out1_fourierTransform_withInception/'
pre_trained_forecaster = tf.saved_model.load(model_path)

In [ ]:
print(checkpoint_filepath)

/content/drive/MyDrive/hobby_project/Models/1SF_nsepy_Model_inp39_out1_vocab90.weights.weights.h5


In [ ]:
transformer.load_weights(checkpoint_filepath)

### Prediction from Pre-trained loaded transformer

In [ ]:
predictions_reconstructed_model = transformer((c_test_pad[:1000],X_test_pad[:1000]), training=False)
y_test_redused = y_test_pad[:1000]

In [ ]:
predictions_val_reconstructed_model = np.argmax(predictions_reconstructed_model, axis=2)
predictions_val_reconstructed_model.shape

(1000, 41)

In [ ]:
i = 1
print('Predictions = \n',predictions_val_reconstructed_model[i])
print('Actuals = \n',y_test_redused[i])

Predictions = 
 [39 38 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39
 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39]
Actuals = 
 [53  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 89]


### Compare saved model with Actuals

In [ ]:
y_reconstructed_model = predictions_val_reconstructed_model[:,1]

In [ ]:
print(y_reconstructed_model.shape)
print(y_test[:1000].shape)

accuracy = accuracy_score(y_test[:1000],y_reconstructed_model)
print(f"Accuracy: {accuracy:.2f}")

(1000,)
(1000, 1)
Accuracy: 0.02


In [ ]:
fig = go.Figure(data=[go.Ohlc(x=np.arange(0,1000,1),
                    open=y_reconstructed_model,
                    high=y_reconstructed_model,
                    low=y_test[:1000].reshape(-1),
                    close=y_test[:1000].reshape(-1)
                              )])
fig.show()

### Exporting the outputs to csv

In [ ]:
data_df = pd.read_csv(path+'/OSF_results.csv')
data_df.columns

Index(['Actuals', 'OSF_Model_1', 'OSF_2'], dtype='object')

In [ ]:
# Uncomment only if you wish to compare the data in excel sheet
data_df = pd.read_csv(path+'/OSF_results.csv')
#data_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

# Using DataFrame.insert() to add a column
#data_df.insert(2, "OSF_2", y_reconstructed_model, True)
data_df.insert(2, "OSF_2_better", y_pred, True)

data_df

,Actuals,OSF_Model_1,OSF_2_better,OSF_2
0,542,542,542,542
1,509,509,509,509
2,722,562,586,588
3,517,517,517,517
4,525,525,525,525
...,...,...,...,...
2518,520,520,520,520
2519,500,500,500,500
2520,542,542,542,542
2521,523,523,523,523


In [ ]:
data_df.to_csv(path+'/OSF_results.csv',index=False)

# Failed Models

# Success Models

In [ ]:
# 1st Best model
# Input formation
'''
data_q1 = data_df.iloc[:13200,:129].copy()
data_q2 = data_df.iloc[:13200,128:257].copy()
data_q3 = data_df.iloc[:13200,256:385].copy()
data_q4 = data_df.iloc[:13200,384:].copy()
data_df_merged = pd.DataFrame()
data_df_merged = pd.concat([pd.DataFrame(data_q1.values),pd.DataFrame(data_q2.values),pd.DataFrame(data_q3.values),pd.DataFrame(data_q4.values)],
                           ignore_index=True)
'''
# Inputs
'''
X Train shape (35640, 128)
Y Train shape (35640, 1)

X Validation shape (11880, 128)
Y Validation shape (11880, 1)

X Test shape (5280, 128)
Y Test shape (5280, 1)
'''
# Hyperparameters
'''
# Hyperparameters for Fourier Transform
signal_length = X_train.shape[1] # Length of the input time series
frame_length = [128, 64, 32] # Hop size for Fourier Transform
frame_size = len(frame_length)
frame_step = [64, 32, 16] # Hop size for Fourier Transform
fft_length = frame_length

# Hyperparameters for Attention Layer and DNN Layer
num_layers = [1, 2, 2] # number of TransformerEncoderLayer layers (Original paper = 6)
num_heads = [1, 1, 2] # number of self-attention heads in the MultiheadAttention layer (Original paper = 8)
dropout_rate = 0.4 # Dropout rate
regularizer_rate = 0.001  # Use to regularizer the weights in attention model, smaller number for less data


# The STFT output shape can be defined as:
# Output Shape = (batch_size, 𝐹, 𝑇)
time_bins = list()
frequency_bins = list()
for i in np.arange(frame_size):
  print(i)
  # Number of Time Frames (T)
  time_bins.append(int(((signal_length - fft_length[i])/frame_step[i] )+1 ))
  print('Time Bins =',time_bins[i])


0
Time Bins = 1
Frequency Bins = 65
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 1 , 65 )

1
Time Bins = 3
Frequency Bins = 33
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 3 , 33 )

2
Time Bins = 7
Frequency Bins = 17
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 7 , 17 )
'''

# Weights
'''
Model: "inception"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ transformer_layer (TransformerLayer) │ ?                           │          78,581 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_1                  │ ?                           │          68,302 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer_2                  │ ?                           │          48,128 │
│ (TransformerLayer)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (3, 1)                      │             385 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
 Total params: 195,396 (763.27 KB)
 Trainable params: 194,352 (759.19 KB)
 Non-trainable params: 1,044 (4.08 KB)
'''

# Output
'''
Test
The Mean Absolute Error of our Model is 2.72
The Root Mean Squared Error of our Model is 27.64
The accuracy of our model is 59.0%